In [1]:
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import json
import re
import pandas as pd
import spacy
from spacy.tokens import Doc, Span
input_file = './../database/corpus/preparation/merged.jsonl'
conll_data = []
# Create a blank Spacy model
nlp = spacy.blank('en')
docs = []  # This will hold the Doc objects
model_name = "./../../../ablation/output/2/"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

with open(input_file, 'r') as f:
    data = [json.loads(line) for line in f]
# For each example in your data
for example in data:
    text = example['text']
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)

    predicted_label_ids = predictions[0].numpy()
    predicted_labels = [model.config.id2label[label_id] for label_id in predicted_label_ids]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    combined_labels = []
    combined_tokens = []
    entity_spans = []

    for token, label in zip(tokens, predicted_labels):
        if token.startswith("##"):
            combined_tokens[-1] = combined_tokens[-1] + token[2:]
        else:
            combined_tokens.append(token)
            combined_labels.append(label)
    # Create a doc with the tokens and spaces
    doc = Doc(nlp.vocab, words=combined_tokens, spaces=[True]*len(combined_tokens))

    # Remove '##' from the text
    #doc_text = ' '.join([token.text.replace(' ##', '') for token in doc])

    entity_spans = []
    for i, label in enumerate(combined_labels):
        l1 = []
        if label != "O":
            span = Span(doc, i, i+1, label)
            entity_spans.append(span)
            l1.append(combined_tokens[i])
            l1.append(label)
            conll_data.append(l1)
    # Assign the entities to the doc
    doc.ents = entity_spans
    docs.append(doc)


/home/cr/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from spacy import displacy
displacy.serve(docs, style='ent')
#html = displacy.render(docs, style='ent', page=True)

In [2]:
def notNaN(num):
    return num == num
def isNaN(num):
    return num != num

def word(f, w):
    f.write("{\"lower\":\"" + w + "\"}")

In [3]:
import pandas as pd

def creatlabelword(label, lTerm):
    doc = pd.read_csv("../database/corpus/labels/"+ label + ".csv", low_memory=False)
    f = open("../database/Label2Train/labels.jsonl","a")
    i=0
    for var in doc["variation"]:
        if(notNaN(doc["RC"][i]) and doc["RC"][i]!="xxx"):
            term = re.split(r'[\s-]', str(doc["RC"][i]))
            for j, w in enumerate(term):
                lTerm.append(w)
        elif(isNaN(doc["RC"][i])):
            term = re.split(r'[\s-]', str(var))

            for j, w in enumerate(term):
                lTerm.append(w)
        i+=1
        
def creatlabelTerm(label, lTerm):
    doc = pd.read_csv("../database/corpus/labels/"+ label + ".csv", low_memory=False)
    f = open("../database/Label2Train/labels.jsonl","a")
    i=0
    for var in doc["variation"]:
        if(notNaN(doc["RC"][i]) and doc["RC"][i]!="xxx"):
            lTerm.append(doc["RC"][i])
        elif(isNaN(doc["RC"][i])):
            lTerm.append(var)
        i+=1

1440


In [6]:
ent = ""
lab = ""

# labels = ["nanoparticle", "property", "material", "event", "technique"]
# lTerm = []
# for label in labels:
#     creatlabelTerm(label, lTerm)
# print(len(lTerm))
# lTerm.append("aunp")
# lTerm.append("aunps")
# lTerm.append("np")
# lTerm.append("nps")
# print(lTerm)
# list1 = set(lTerm)
# print(len(list1), list1)

with open("./../database/ablation/2/listvar2.txt", 'r') as f:
    lTerm = [line.strip() for line in f]
with open("./../database/ablation/2/listDelete2.txt", 'r') as f:
    ldel = [line.strip() for line in f]

flag = 0
listNewTerm = []
lab = conll_data[0][1][2:]

for i in range(len(conll_data)):
    if(conll_data[i][1][:1]=="B"):
        if(ent not in lTerm and ent!="[SEP]"):
            listNewTerm.append([ent, lab])
            print(f"{ent}: {lab}")
        ent = conll_data[i][0]
    else:
        if(conll_data[i][0]=="-" or ent[-1]=="-"):
            ent = ent + conll_data[i][0]
        else:
            ent = ent + " " + conll_data[i][0]
    lab = conll_data[i][1][2:]
listNewTerm = list(set(tuple(sublist) for sublist in listNewTerm))
listNewTerm = [list(sublist) for sublist in listNewTerm]


# 创建 pandas DataFrame
df = pd.DataFrame(listNewTerm, columns=["Entity", "Label"])

lrf = []

for i in range(len(df["Entity"])):
    ent = df["Entity"][i]
    if df["Entity"][i] in ldel:
        lab = df["Label"][i]
        lrf.append([ent,lab])
dfrefound = pd.DataFrame(lrf, columns=["Entity", "Label"])
dfrefound.to_csv("./../database/ablation/2/refound.csv", index=False)
    
# 将 DataFrame 写入 CSV 文件
df.to_csv("newterms.csv", index=False)



: nanoparticle
biocompatibility: property
anticancer: property
hydrophilic: property
folic acid: material
folic acid: material
folic acid: material
folic acid: material
folic acid: material
folic acid: material
anticancer: property
water insoluble: property
anticancer: property
anticancer: property
carboxylic acid: material
hydrophilic: property
anticancer: property
folic acid: material
folic acid: material
hydrophilic: property
cysteine: material
hydrophilicity: property
hydrophilic: property
anticancer: property
hydrophilicity: property
multi-walled nanotubes: nanoparticle
gold nanoparticle: nanoparticle
gold nanoparticle: nanoparticle
vehicles: material
vehicles: material
vehicles: material
vehicles: material
hydrophilic: property
hydrophilicity: property
anticancer: property
anticancer: property
anticancer: property
core shell: nanoparticle
steroidal anti-inflammatory: property
folic acid: material
folic acid: material
anticancer: property
anticancer: property
anticancer: property


In [37]:
ent = ""
lab = ""

labels = ["nanoparticle", "property", "material", "event", "technique"]
lTerm = []
for label in labels:
    creatlabelword(label, lTerm)
print(len(lTerm))
lTerm.append("aunp")
lTerm.append("aunps")
lTerm.append("np")
lTerm.append("nps")
print(lTerm)
list1 = set(lTerm)
print(len(list1), list1)

flag = 0
listNewTerm2 = []
for i in range(len(conll_data)):
    if(conll_data[i][1][:1]=="B"):
        if(flag):
            l0 = []
            l0.append(ent)
            l0.append(lab)
            listNewTerm2.append(l0)        
        ent = conll_data[i][0]
        flag = 0
    else:
        if(conll_data[i][0] not in lTerm and conll_data[i][0]!="-"):
            ent = ent + " " + conll_data[i][0]
            lab = conll_data[i][1][2:]
            flag=1
        else:
            ent = ent + " " + conll_data[i][0]
# 创建 pandas DataFrame
df = pd.DataFrame(listNewTerm2, columns=["Entity", "Label"])

# 将 DataFrame 写入 CSV 文件
df.to_csv("newwords.csv", index=False)


3202
['nanoparticle', 'nanoparticles', 'nanospheres', 'nanosphere', 'nanocage', 'nanocages', 'nanocapsule', 'nanocapsulates', 'nanocapsules', 'nanoshell', 'nanoshells', 'nanorods', 'nanorod', 'nanotubes', 'nanotube', 'fullerene', 'fullerenes', 'fulleren', 'liposomes', 'liposomal', 'liposome', 'liposoms', 'liposom', 'plga', 'nanoparticles', 'biodegradable', 'polymeric', 'nanoparticles', 'biodegradable', 'nanoparticles', 'chitosan', 'nanoparticles', 'chitosan', 'nanoparticle', 'core', 'shell', 'nanoparticles', 'core', 'shell', 'nanoparticle', 'core', 'shell', 'gold', 'palladium', 'nanoparticles', 'drug', 'loaded', 'polymer', 'nanoparticles', 'drug', 'loaded', 'nanoparticles', 'drug', 'loaded', 'upconversion', 'nanoparticles', 'fluorescent', 'barcode', 'nanoparticles', 'fluorescent', 'carbon', 'nanoparticles', 'fluorescent', 'nanoparticles', 'fluorescent', 'organic', 'nanoparticles', 'fluorescent', 'silver', 'nanoparticle', 'fluorescent', 'gold', 'nanoparticles', 'fluorescent', 'semicondu

In [22]:
for i in range(len(conll_data)):
    if(conll_data[i][0] not in lTerm):
        print(f"{conll_data[i][0]}: {conll_data[i][1]}")

alkaloid: B-material
[SEP]: B-material
[SEP]: B-material
antivirals: B-property
multi: B-nanoparticle
-: I-nanoparticle
walled: I-nanoparticle
nanofilaments: B-nanoparticle
anti: B-property
-: I-property
inflammatory: I-property
non: B-property
-: I-property
steroidal: I-property
anti: I-property
-: I-property
inflammatory: I-property
anti: B-property
-: I-property
inflammatory: I-property
anti: B-property
-: I-property
inflammatory: I-property
anti: B-property
-: I-property
inflammatory: I-property
5: B-material
-: I-material
fluorouracil: I-material
5: B-material
-: I-material
fluorouracil: I-material
[SEP]: B-material
anti: B-property
-: I-property
inflammatory: I-property
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
molecule: I-property
regulators: I-property
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
[SEP]: B-material
nanopeptides: B-material
nanopolymers